In [24]:
%pip install -qU langchain langchain-openai langchain-community langchain-experimental pandas==2.2.2

In [25]:
!wget https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv -O titanic.csv

--2024-10-09 06:37:26--  https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv
Resolving web.stanford.edu (web.stanford.edu)... 171.67.215.200, 2607:f6d0:0:925a::ab43:d7c8
Connecting to web.stanford.edu (web.stanford.edu)|171.67.215.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44225 (43K) [text/csv]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  43.19K  --.-KB/s    in 0.1s    

2024-10-09 06:37:26 (312 KB/s) - ‘titanic.csv’ saved [44225/44225]



In [26]:
import pandas as pd

df = pd.read_csv("titanic.csv")
print(df.shape)
print(df.columns.tolist())

(887, 8)
['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Fare']


In [28]:
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

engine = create_engine("sqlite:///titanic.db")
df.to_sql("titanic2", engine, index=False)

887

In [29]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
print(db.run("SELECT * FROM titanic2 WHERE Age > 60;"))

sqlite
['titanic', 'titanic2']
[(0, 2, 'Mr. Edward H Wheadon', 'male', 66.0, 0, 0, 10.5), (0, 1, 'Mr. Engelhart Cornelius Ostby', 'male', 65.0, 0, 1, 61.9792), (0, 1, 'Mr. Albert A Stewart', 'male', 64.0, 0, 0, 27.7208), (0, 1, 'Mr. George B Goldschmidt', 'male', 71.0, 0, 0, 34.6542), (0, 3, 'Mr. Patrick Connors', 'male', 70.5, 0, 0, 7.75), (0, 1, 'Mr. Wyckoff Van der hoef', 'male', 61.0, 0, 0, 33.5), (0, 1, 'Mr. William Thomas Stead', 'male', 62.0, 0, 0, 26.55), (1, 1, 'Miss. Kornelia Theodosia Andrews', 'female', 63.0, 1, 0, 77.9583), (0, 3, 'Mr. Frank Duane', 'male', 65.0, 0, 0, 7.75), (0, 3, 'Mr. Johan Hansen Nysveen', 'male', 61.0, 0, 0, 6.2375), (0, 1, 'Mr. Mark Fortune', 'male', 64.0, 1, 4, 263.0), (0, 1, 'Mr. Francis Davis Millet', 'male', 65.0, 0, 0, 26.55), (1, 3, 'Mrs. (Hedwig) Turkula', 'female', 63.0, 0, 0, 9.5875), (0, 1, 'Mr. Ramon Artagaveytia', 'male', 71.0, 0, 0, 49.5042), (0, 3, 'Mr. James Webber', 'male', 66.0, 0, 0, 8.05), (0, 3, 'Mr. Samuel Beard Risien', 'male', 

In [30]:
!pip install -qU langchain-groq

In [31]:
import os
os.environ["GROQ_API_KEY"] = "YOUR API KEY"

In [32]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="llama3-8b-8192")

In [33]:
from langchain_community.agent_toolkits import create_sql_agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [34]:
agent_executor.invoke({"input": "what's the average age of survivors in titanic2"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{'tool_input': ''}`
responded:  

I'm going to start by calling the "sql_db_list_tables" tool to get a list of tables in the database. 



titanic, titanic2
Invoking: `sql_db_schema` with `{'table_names': 'titanic'}`



CREATE TABLE titanic (
	"Survived" BIGINT, 
	"Pclass" BIGINT, 
	"Name" TEXT, 
	"Sex" TEXT, 
	"Age" FLOAT, 
	"Siblings/Spouses Aboard" BIGINT, 
	"Parents/Children Aboard" BIGINT, 
	"Fare" FLOAT
)

/*
3 rows from titanic table:
Survived	Pclass	Name	Sex	Age	Siblings/Spouses Aboard	Parents/Children Aboard	Fare
0	3	Mr. Owen Harris Braund	male	22.0	1	0	7.25
1	1	Mrs. John Bradley (Florence Briggs Thayer) Cumings	female	38.0	1	0	71.2833
1	3	Miss. Laina Heikkinen	female	26.0	0	0	7.925
*/
Invoking: `sql_db_query` with `{'query': 'SELECT AVG(Age) FROM titanic WHERE Survived = 1'}`


[(28.408391812865496,)]The average age of survivors in the Titanic dataset is 28.4 years.

> Finished chain.


{'input': "what's the average age of survivors in titanic2",
 'output': 'The average age of survivors in the Titanic dataset is 28.4 years.'}

In [36]:
# testing code generation abilities of LLMs

ai_msg = llm.invoke("I have a dataframe 'df' with columns 'Age' and 'Fare', Write the python code to test correlation of the same, output only the markdown of the python code and nothing else.")
print(ai_msg.content)

```
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

corr_coef, p_val = pearsonr(df['Age'], df['Fare'])
print(f"Correlation Coefficient: **{corr_coef:.2f}**")
print(f"P-value: **{p_val:.4f}**")
```


In [37]:
import pandas as pd
from langchain_experimental.tools import PythonAstREPLTool
from langchain_core.prompts import ChatPromptTemplate

df = pd.read_csv("titanic.csv")
tool = PythonAstREPLTool(locals={"df": df})
tool.invoke("df.columns")

Index(['Survived', 'Pclass', 'Name', 'Sex', 'Age', 'Siblings/Spouses Aboard',
       'Parents/Children Aboard', 'Fare'],
      dtype='object')

In [41]:
tool.invoke("(df.head(3))")

,Survived,Pclass,Name,Sex,Age,Siblings/Spouses Aboard,Parents/Children Aboard,Fare
0,0,3,Mr. Owen Harris Braund,male,22.0,1,0,7.2500
1,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cum...,female,38.0,1,0,71.2833
2,1,3,Miss. Laina Heikkinen,female,26.0,0,0,7.9250


In [42]:
tool.name

'python_repl_ast'

In [44]:
llms_with_tools = llm.bind_tools([tool], tool_choice=tool.name)
response = llms_with_tools.invoke("I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2dnw', 'function': {'arguments': '{"query":"from scipy.stats import pearsonr; pearsonr(df[\'Age\'], df[\'Fare\'])[0]"}', 'name': 'python_repl_ast'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 95, 'prompt_tokens': 875, 'total_tokens': 970, 'completion_time': 0.079166667, 'prompt_time': 0.117544881, 'queue_time': 0.0028056659999999983, 'total_time': 0.196711548}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-c4e9c8eb-ca12-4ca0-af5e-0ddc9fd0990f-0', tool_calls=[{'name': 'python_repl_ast', 'args': {'query': "from scipy.stats import pearsonr; pearsonr(df['Age'], df['Fare'])[0]"}, 'id': 'call_2dnw', 'type': 'tool_call'}], usage_metadata={'input_tokens': 875, 'output_tokens': 95, 'total_tokens': 970})

In [45]:
response.tool_calls

[{'name': 'python_repl_ast',
  'args': {'query': "from scipy.stats import pearsonr; pearsonr(df['Age'], df['Fare'])[0]"},
  'id': 'call_2dnw',
  'type': 'tool_call'}]

In [47]:
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)
(llms_with_tools | parser).invoke(
    "I have a dataframe 'df' and want to know the correlation between the 'Age' and 'Fare' columns"
)


{'query': "import pandas as pd; import numpy as np; corr = df[['Age', 'Fare']].corr(); print(corr.iloc[0,1])"}

In [50]:
system = f"""You have access to a pandas dataframe `df`. \
Here is the output of `df.head().to_markdown()`:

\`\`\`
{df.head().to_markdown()}
\`\`\`

Given a user question, write the Python code to answer it. \
Return ONLY the valid Python code and nothing else. \
Don't assume you have access to any libraries other than built-in Python ones and pandas."""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])
code_chain = prompt | llms_with_tools | parser
code_chain.invoke({"question": "What's the correlation between age and fare"})


{'query': "df.corr('Age', 'Fare').round(2)"}

In [52]:
from langchain_experimental.agents import create_pandas_dataframe_agent

agent = create_pandas_dataframe_agent(llm, df, agent_type="openai-tools", verbose=True, allow_dangerous_code=True)
agent.invoke(
    {
        "input": "What's the correlation between age and fare? is that greater than the correlation between fare and survival?"
    }
)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df.corr()['Age'].max()"}`
responded: Let's use the tools provided to find the answer.

Here is the JSON string:


ValueError: could not convert string to float: 'Mr. Owen Harris Braund'
Invoking: `python_repl_ast` with `{'query': "df.corr()['Fare'].max()"}`
responded: Let's use the tools provided to find the answer.

Here is the JSON string:


ValueError: could not convert string to float: 'Mr. Owen Harris Braund'
Invoking: `python_repl_ast` with `{'query': "df.corr()['Age']['Fare']"}`
responded: Let's use the tools provided to find the answer.

Here is the JSON string:


ValueError: could not convert string to float: 'Mr. Owen Harris Braund'
Invoking: `python_repl_ast` with `{'query': "df[['Age', 'Fare']].corr()"}`
responded: It seems like the issue is that the dataframe contains non-numeric data, specifically the 'Name' column, which is causing the error.

To calculate the correlation between age an

{'input': "What's the correlation between age and fare? is that greater than the correlation between fare and survival?",
 'output': 'The correlation between age and fare is 0.112329, and the correlation between fare and survival is not applicable because fare is not a direct factor in survival.'}

In [53]:
df_1 = df[["Age", "Fare"]]
df_2 = df[["Fare", "Survived"]]

tool = PythonAstREPLTool(locals={"df_1": df_1, "df_2": df_2})
llm_with_tool = llm.bind_tools(tools=[tool], tool_choice=tool.name)
df_template = """\`\`\`python
{df_name}.head().to_markdown()
>>> {df_head}
\`\`\`"""
df_context = "\n\n".join(
    df_template.format(df_head=_df.head().to_markdown(), df_name=df_name)
    for _df, df_name in [(df_1, "df_1"), (df_2, "df_2")]
)

system = f"""You have access to a number of pandas dataframes. \
Here is a sample of rows from each dataframe and the python code that was used to generate the sample:

{df_context}

Given a user question about the dataframes, write the Python code to answer it. \
Don't assume you have access to any libraries other than built-in Python ones and pandas. \
Make sure to refer only to the variables mentioned above."""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{question}")])

chain = prompt | llm_with_tool | parser | tool
chain.invoke(
    {
        "question": "return the difference in the correlation between age and fare and the correlation between fare and survival"
    }
)

0.11232863699941621